In [51]:
import numpy as np
import pandas as pd
import pickle

In [53]:
# Text Pre-Processing
import re
import string

In [55]:
txt = 'great product, I like it'

In [57]:
# Remove Punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation, '')
    return text

In [59]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [61]:
# load trained model
with open('../static/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [63]:
# Stemming (get base word)
from nltk.stem import PorterStemmer
ps = PorterStemmer()


In [65]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
#convert uppercase to lowercase
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# remove links using regression
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
# Remove numbers
    data['tweet'] = data['tweet'].str.replace(r'\d+', '', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [67]:
preprocessedText = preprocessing(txt)

In [69]:
preprocessedText

0    great product like
Name: tweet, dtype: object

In [71]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [73]:
# should convert text values into numerical values (Vectorization)
def vectorizer(ds, vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)

    return vectorized_lst_new

In [75]:
vectorized_txt = vectorizer(preprocessedText, tokens)

In [77]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [79]:
model.predict(vectorized_txt)

array([0], dtype=int64)